# Transfer Learning with TensorFlow Part 2: Fine-tuning

In the previous notebook, we covered transfer learning feature extraction, now it's time to learn about a new kind of transfer-learning: fine-tuning

## Creating helper functions

In previous notebooks, we have created a bunch of helper functions, now we could rewrite them all, however, this is tedious.

So, it's a good idea to put functions you will want to use again in a script you can download and import into your notebooks (or elsewhere).

We have done this for some of the functions we have used previously.

In [15]:
import sys
import os

# Subir dos niveles: desde notebooks/pruebas → mi_proyecto/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Ahora puedes importar
from utils.helper_functions import create_tensorboard_callback, plot_loss_curves, walk_through_dir